In [1]:
import pandas as pd

In [2]:
rules = pd.read_json("../data/arules-10000-00035.json")

In [3]:
rules["antecedents"] = rules["antecedents"].apply(lambda x: frozenset(x))
rules["consequents"] = rules["consequents"].apply(lambda x: frozenset(x))

In [4]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Cringetopia),(nextfuckinglevel),0.044759,0.079712,0.009866,0.220419,2.765198,0.006298,1.180491
1,(nextfuckinglevel),(Cringetopia),0.079712,0.044759,0.009866,0.123766,2.765198,0.006298,1.090167
2,(funny),(Cringetopia),0.079742,0.044759,0.006839,0.085769,1.916250,0.003270,1.044857
3,(Cringetopia),(funny),0.044759,0.079742,0.006839,0.152806,1.916250,0.003270,1.086242
4,(pics),(Cringetopia),0.072449,0.044759,0.006083,0.083960,1.875842,0.002840,1.042794
...,...,...,...,...,...,...,...,...,...
3963457,(OnlyfansAddict),"(onlyfanssubscribe, HotOnlyfans, OnlyfansXXX, ...",0.001483,0.000424,0.000363,0.244898,578.029155,0.000363,1.323763
3963458,(OnlyfansXXX),"(OnlyfansAddict, onlyfanssubscribe, HotOnlyfan...",0.002239,0.000363,0.000363,0.162162,446.540541,0.000362,1.193115
3963459,(onlyfans_get_noticed),"(OnlyfansAddict, onlyfanssubscribe, HotOnlyfan...",0.001634,0.000393,0.000363,0.222222,564.854701,0.000363,1.285208
3963460,(OnlyfansAmateurs),"(OnlyfansAddict, onlyfanssubscribe, HotOnlyfan...",0.002028,0.000363,0.000363,0.179104,493.194030,0.000362,1.217739


In [5]:
def get_rules(set_of_subreddits, top_n=100):
    antecedents_rule =  rules['antecedents'].apply(lambda x: set_of_subreddits.issuperset(x))
    new_rules =  rules[antecedents_rule].copy()
#     new_rules["Coefficient"] = new_rules["confidence"] + new_rules["support"]
    new_rules["Coefficient"] = new_rules["lift"]
#     new_rules["Coefficient"] = new_rules["confidence"] - new_rules["support"] + new_rules["lift"] + new_rules["leverage"]+ new_rules["conviction"]
#     new_rules["consequents"] = new_rules["consequents"].apply(lambda x: x - set_of_subreddits - OBVIOUS_SUBREDDITS)
    new_rules["consequents"] = new_rules["consequents"].apply(lambda x: x - set_of_subreddits)
    new_rules = new_rules[new_rules["consequents"].apply(lambda x: len(x) > 0)]
    if len(new_rules) == 0:
        return []
    new_rules = new_rules[['consequents', "Coefficient"]]
    new_rules = new_rules.explode("consequents")
    new_rules["consequents"] = new_rules["consequents"].apply(lambda x:list(x)[0])
    new_rules = new_rules.groupby("consequents")["Coefficient"].max().reset_index()    
    return list(new_rules.nlargest(top_n, "Coefficient")["consequents"])

In [6]:
df = get_rules({ 'DestinyTheGame', 'Diablo', 'CitiesSkylines', 'CODWarzone', 'zelda', 'witcher'})
df

['DestinyLore',
 'CrucibleGuidebook',
 'LowSodiumDestiny',
 'Fireteams',
 'raidsecrets',
 'Witcher3',
 'DestinyFashion',
 'aww',
 'funny',
 'destiny2',
 'Breath_of_the_Wild',
 'mildlyinteresting',
 'dankmemes',
 'Warzone',
 'DestinyMemes',
 'modernwarfare',
 'gaming',
 'blackopscoldwar',
 'CODZombies',
 'outriders',
 'Warframe',
 'cyberpunkgame',
 'pokemon',
 'NintendoSwitch',
 'halo',
 'xboxone',
 'titanfall',
 'XboxSeriesX',
 'Rainbow6',
 'Overwatch',
 'Battlefield',
 'pcmasterrace',
 'apexlegends',
 'PS4',
 'Eldenring',
 'Games',
 'PS5',
 'LivestreamFail',
 'leagueoflegends',
 'MadeMeSmile',
 'Damnthatsinteresting',
 'Cringetopia',
 'iamatotalpieceofshit',
 'movies',
 'pics',
 'oddlysatisfying',
 'Unexpected',
 'TooAfraidToAsk',
 'facepalm',
 'nextfuckinglevel',
 'todayilearned',
 'HolUp',
 'PrequelMemes',
 'television',
 'technology',
 'mildlyinfuriating',
 'Whatcouldgowrong',
 'Showerthoughts',
 'LifeProTips',
 'IdiotsInCars',
 'starterpacks',
 'trashy',
 'videos',
 'WhitePeopleTw

In [7]:
df = get_rules({ 'Bitcoin'})
df

['BitcoinBeginners',
 'BitcoinMarkets',
 'CryptoMarkets',
 'btc',
 'ethereum',
 'Buttcoin',
 'CryptoCurrencies',
 'CoinBase',
 'ethtrader',
 'XRP',
 'binance',
 'cardano',
 'SatoshiStreetBets',
 'CryptoMoonShots',
 'SafeMoon',
 'investing',
 'SHIBArmy',
 'Economics',
 'StockMarket',
 'stocks',
 'Wallstreetbetsnew',
 'ProgrammerHumor',
 'Conservative',
 'personalfinance',
 'conspiracy',
 'dataisbeautiful',
 'GME',
 'funny',
 'science',
 'gifs',
 'technology',
 'space',
 'coolguides',
 'mildlyinteresting',
 'europe',
 'Coronavirus',
 'Wellthatsucks',
 'WTF',
 'Futurology',
 'explainlikeimfive',
 'PoliticalHumor',
 'NatureIsFuckingLit',
 'todayilearned',
 'Showerthoughts',
 'JusticeServed',
 'Whatcouldgowrong',
 'WhitePeopleTwitter',
 'nottheonion',
 'gaming',
 'OldSchoolCool',
 'MurderedByWords',
 'pics',
 'videos',
 'pcmasterrace',
 'LifeProTips',
 'facepalm',
 'nextfuckinglevel',
 'Unexpected',
 'HolUp',
 'trashy',
 'Damnthatsinteresting',
 'IdiotsInCars',
 'WinStupidPrizes',
 'iamatot

In [8]:
test_users = [
{"gothsluts": 16, "2000sGirls": 33, "nj4nj": 1, "Yololaceandlingerie": 1, "u_Unknownpleasures89": 1, "TeenBeauties": 1, "assinthong": 1, "18_19": 1, "GirlsGoneWilderness": 1, "collegesluts": 10, "GaybrosGoneWild": 1, "facesitting": 1, "milf": 3, "cougars_and_milfs_sfw": 3, "bigasses": 1},
{"flatearth": 74, "teenagers": 1},
{"HotOnlyfans": 19, "onlyfanschicks": 17, "FreeOnlyFansPage": 17, "HornyOnlyfans": 4, "onlyfansgirls101": 16, "AdorableOnlyfans": 2},
{"CallOfDutyMobile": 75},
{"Amd": 57, "battlefield2042": 1, "Battlefield6": 1, "pathofexile": 15, "PathOfExileBuilds": 1},
{"AskComputerScience": 11, "compsci": 4, "learnmachinelearning": 13, "computerscience": 10, "deeplearning": 6, "learnprogramming": 5, "AskPhysics": 6, "AskProgramming": 7, "Chodi": 1, "algorithms": 2, "linuxquestions": 1, "MachineLearning": 1, "math": 4, "crypto": 3, "softwarearchitecture": 1},
{"starwarsmemes": 1, "nextfuckinglevel": 9, "darkjokes": 1, "juggling": 15, "AskReddit": 4, "MetalMemes": 5, "Unexpected": 2, "trackandfield": 4, "okbuddyretard": 8, "cursedcomments": 3, "Meshuggah": 2, "mildlyinfuriating": 3, "memes": 11, "interestingasfuck": 1, "hermitcraftmemes": 1, "ConservativeMemes": 1, "BrandNewSentence": 2, "oddlyterrifying": 1, "ChoosingBeggars": 1},
{"NoFap": 75},
{"UFOscience": 13, "UFOs": 26, "ufomemes": 1, "UAP": 2, "ufo": 15, "HighStrangeness": 5, "space": 6, "CoronavirusDownunder": 2, "UFObelievers": 2, "australia": 1, "UF0": 1, "brexit": 1}
]

In [9]:
for user in test_users:
    sub_red = {k for k,v in user.items()}
    print("User likes:", sub_red)
    print("User should like:",get_rules(sub_red,10))

User likes: {'2000sGirls', 'TeenBeauties', 'GaybrosGoneWild', '18_19', 'facesitting', 'cougars_and_milfs_sfw', 'Yololaceandlingerie', 'u_Unknownpleasures89', 'milf', 'collegesluts', 'GirlsGoneWilderness', 'assinthong', 'bigasses', 'gothsluts', 'nj4nj'}
User should like: ['BiCuriousGuysChat', 'adorableporn', 'MassiveCock', 'gaymersgonewild', 'cock', 'LegalTeens', 'Balls', 'gaynsfw', 'ThickDick', 'ratemycock']
User likes: {'flatearth', 'teenagers'}
User should like: []
User likes: {'FreeOnlyFansPage', 'HornyOnlyfans', 'HotOnlyfans', 'onlyfanschicks', 'AdorableOnlyfans', 'onlyfansgirls101'}
User should like: ['OnlyfansXXX', 'GoneWildOnlyfans', 'onlyfans_get_noticed', 'onlynudechicks', 'Onlyfanspromoss', 'JuicyOnlyfans', 'naughtychicks', 'OnlyFansPromotions', 'OnlyFansBusty', 'onlyfanshottest']
User likes: {'CallOfDutyMobile'}
User should like: []
User likes: {'PathOfExileBuilds', 'Battlefield6', 'Amd', 'battlefield2042', 'pathofexile'}
User should like: ['Battlefield', 'AyyMD', 'intel', '